# hands-on implementation of cnn with numpy-------conv layer
> use two method 
- naive method
- fast matrix compute

## basic components
- conv_layer
- pooling layer
- activation layer(relu)
- fully connected layer
- output layer(softmax for classification problem)

## faster conv opt
> 这里主要使用的就是im2col优化方法：通过将图像展开，使得卷积运算可以变成两个矩阵乘法

![conv_img](https://pic1.zhimg.com/80/v2-24557fb0729cacb1e70d30a1fc489150_hd.jpg)
> reference:   https://hal.inria.fr/file/index/docid/112631/filename/p1038112283956.pdf


## conv_layer implementation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline
print(np.__version__)

## conv2d basic params
- 输入数据的shape = [N,W,H,C] N=Batchsize/W=width/H=height/C=channels
- 卷积核的尺寸ksize ,个数output_channels, kernel shape [output_channels,k,k,C]
- 卷积的步长stride，基本默认为1.
- 卷积的方法，VALID or SAME，即是否通过padding保持输出图像与输入图像的大小不变(stride==1时候)
- $$\frac{\partial loss}{\partial conv\_out}$$ 
- $$\frac{\partial loss}{\partial filter},\frac{\partial loss}{\partial bias}$$
- 已上分别用self.delta self.filter_grad self.bias_grad

## analyse forward + backward propagation
- forward propagation = x\*w (\*表示卷积操作) -> output

- backward propagation = **padded** output_delta\* **flipped** w(\*表示卷积)->input_delta
> 实现网络中的一个操作的反向传播，基本操作只需要完成以上这两点
> - 1是保证导数信息（残差）继续在网络中向前传播. 
> - 2是根据传入的导数信息求解自身参数的导数。

- backward propagation 
    - weights
        - filter_1 $$\frac{\partial (Output\_features)}{\partial (Kernel)} = \frac{\partial ((Input\_features)*(Kernel))}{\partial (Kernel)} = Input\_features.T$$
        - filter_2 $$\frac{\partial (loss)}{\partial (Kernel)} = Input\_features.T * \frac{\partial (loss)}{\partial (Output\_features)}$$
        - bias $$\frac{\partial (loss)}{\partial (Bias)} = \sum_{axis=0}(\frac{\partial{loss}}{\partial{Output\_features}})$$

    - delta 
        - 因为im2col之后的Input Features矩阵和Input本身不是一一对应的（单射) 所以不能用之前的矩阵形式公式进行计算
        - 由于只需要统计有贡献的元素 通过卷积过程元素对应情况 结合图像以及公示
        - $$\frac{\partial loss}{\partial A} = \sum _{x=a,...,i} delta_x * fliped\_kernel[index(delta_x)]$$
        - **final conclusion**: **卷积层输入的delta计算 等于 卷积层输出的delta与翻转后的Kernel进行卷积(deconv)**
        - NOTE： 在dconv过程中需要注意padding(stride,padding，边角问题)
            - 如果是stride!=1 需要在内部进行padding (行列间padding)
            - 如果stride==1 只需要在边角padding
            - 其中边角padding有
                - same : **ksize/2**
                - valid : **ksize-1**
            - 无论怎样对于delta进行padding 但是在dconv过程需要使用**stride=1**
        - reference：
            - https://zhuanlan.zhihu.com/p/33802329
            - https://www.jianshu.com/p/b6b9a1b0aabf
            - 上面两个讲反向传播很好 一个代码 一个公式 
    
    - NOTE:代码实现多一维batchsize，一个batch里面的w_gradient和b_gradient则是对该batch求和得到的。

In [50]:
class Conv2D(object):
    def __init__(self,input_shape,output_channels,ksize,stride=1,padding='same'):
        self.input_shape = input_shape
        self.input_channels = input_shape[-1]
        self.output_channels = output_channels
        self.ksize = ksize
        self.stride = stride
        self.padding = padding
        # default batch_size = shape[0]
        self.batch_size = input_shape[0]
        
        # initialize params with standardnormal + scaler
        # NOTE:msra方法设置scaler
        weight_scaler = math.sqrt(ksize**2*self.input_channels/2)
        self.filter = np.random.randn(self.output_channels,ksize,ksize,self.input_channels)/weight_scaler
        self.bias = np.random.randn(self.output_channels)/weight_scaler
        
        # back propagation params
        shape = self.input_shape
        if padding == 'valid':
            self.delta = np.zeros((shape[0],(shape[1]-ksize+1)//self.stride,
                                  (shape[2]-ksize+1)//self.stride,self.output_channels))
        elif padding == 'same':
            self.delta = np.zeros((shape[0],shape[1]//self.stride,
                                   shape[2]//self.stride,self.output_channels))   
        else:
            raise AttributeError('unsupported padding method str')
        self.output_shape = self.delta.shape
        print(self.output_shape)
        self.filter_grad = np.zeros(self.filter.shape)
        self.bias_grad = np.zeros(self.bias.shape)
    def _conv(self,x):
        """
        faster convolution with matrix method than naive dot product(element-wise)
        four steps:
        1.reshape filter
        2.padding
        3.im2col save image
        4.matmul(input_features,kernel_matrix)=output_features
        """
        # shape=(self.output_channels,ksize,ksize,self.input_channels)
        col_filter = np.transpose(self.filter,[1,2,3,0])
        col_filter = col_filter.reshape([-1,self.output_channels])
        if self.padding == 'same':
            x = np.pad(x,((0,0),(self.ksize//2,self.ksize//2),(self.ksize//2,self.ksize//2),(0,0)),
                      mode='constant',constant_values = 0)
        # 整个batch一起处理
        #self.img_cols = self._img2col(x)

        # 每个sample in batch 分别处理
        self.img_cols = []
        self.conv_out = []
        for i in range(self.batch_size):
            img_i = x[i][np.newaxis,:] # 保障4dim
            nowcol = self._img2col(img_i,self.ksize,self.stride)
            self.img_cols.append(nowcol)
            self.conv_out.append(np.reshape(
                np.dot(nowcol,col_filter)+self.bias,
                self.delta[0].shape))
            
        self.img_cols = np.array(self.img_cols)
        self.conv_out = np.array(self.conv_out)
        return self.conv_out
    
    def _img2col(self,image,ksize,stride):
        """tool function of transform img-> matrix
        only to one sample
        otherwise:reshape 等操作需要改变处理方式
        """
        # image is a 4d tensor([batchsize, width ,height, channel])
        img_cols = []
        for i in range(0,image.shape[1]-ksize+1,stride):
            for j in range(0,image.shape[2]-ksize+1,stride):
                nowcol = image[:,i:i+ksize,j:j+ksize,:].reshape([-1])
                img_cols.append(nowcol)
        img_cols = np.array(img_cols)
        return img_cols
    
    def forward_propagate(self,x):
        return self._conv(x)
    
    def _dconv(self):
        """
        deconv of padded eta with flippd kernel to get next_eta
        """
        if self.padding == 'valid':
            pad_delta = np.pad(self.delta,
                              ((0,0),(self.ksize-1,self.ksize-1),(self.ksize-1,self.ksize-1),(0,0)),
                              mode='constant',constant_values=0)
            
        elif self.padding == 'same':
            pad_delta = np.pad(self.delta,
                              ((0,0),(self.ksize//2,self.ksize//2),(self.ksize//2,self.ksize//2),(0,0)),
                              mode='constant',constant_values=0)
        # only to 0,1 dims (fliplr,flipud)
        # 使用swapaxes与transpose类似功能 but只能交换两个维度
        # (kszie,ksize,output_channels,input_channels)
        flipped_filter = np.transpose(self.filter,[1,2,0,3])
        flipped_filter = np.fliplr(np.flipud(flipped_filter))
        col_flipped_filter = flipped_filter.reshape([-1,self.input_channels])
        # delta img2col with ** list generator **
        col_pad_delta = np.array(
            [self._img2col(pad_delta[i][np.newaxis,:],
                           self.ksize,self.stride) for i in range(self.batch_size)])
        # dconv (matmul)
        input_delta = np.dot(col_pad_delta,col_flipped_filter)
        # 直接reshape就可以实现 因为已经分开batch处理了
        input_delta = input_delta.reshape(self.input_shape)
        return input_delta
    
    def update_params(self,learning_rate=1e-5, weight_decay=1e-4):
        """
        use L2 regularization 
        并且使用了新的更新方式 ： simple sgd + weight decay
        """
        # weight_decay = L2 regularization
        self.filter *= (1 - weight_decay)
        self.bias *= (1 - weight_decay)
        self.filter -= learning_rate * self.filter_grad
        self.bias -= learning_rate * self.bias_grad
        
    def backward_propagate(self,delta,learning_rate=1e-5, weight_decay=1e-4):
        """
        deconv of padded delta with flippd kernel to get next_delta
        four steps:
        1.对输入的delta根据method进行padding操作。
        2.对卷积核参数进行翻转。
        3.对卷积核进行转置操作。(小心 numpy运行的未必是想象的结果)
        4.对翻转后的卷积核与padding后的pad_delta进行卷积计算，方法同conv.forward
        """
        self.delta = delta
        # reshape
        col_delta = np.reshape(self.delta,
                              [self.batch_size,-1,self.output_channels])
        
        # 还原 保障
        self.filter_grad = np.zeros(self.filter.shape)
        self.bias_grad = np.zeros(self.bias.shape)
        
        # weights gradient update
        for i in range(self.batch_size):
            self.filter_grad += np.dot(self.img_cols[i].T,
                                       col_delta[i]).reshape(self.filter.shape) 
        self.bias_grad+=np.sum(col_delta,axis=(0,1))
        
        self.update_params(learning_rate,weight_decay)

        
        # dconv to backpropagation grad
        return self._dconv()
    
    

In [ ]:
# print(np.random.standard_normal((1,2)))
# print(np.random.randn(1,2))
# demo
old = np.array([[[1],[3]],[[1],[3]],[[1],[3]]])
print(old.shape)
new = np.transpose(old,axes=[0,2,1])
print(new.shape)
print(new)
new = np.transpose(old,axes=[2,1,0])
print(new.shape)
new = np.transpose(old,axes=[1,0,2])
print(new.shape)
print(np.reshape([[1,2],[3,4]],[-1]).shape)
np.array([[1,2],[3,4]])
a = np.array([1,2,3])
b = a.reshape([1,-1])
print(a.shape,b.shape)
print(a is b)

In [53]:
# test conv layer
if __name__ == '__main__':
    img = np.ones((2,32,32,3))
    conv = Conv2D(img.shape,12,3,1,'same')
    conv_out = conv.forward_propagate(img)
    # 模拟loss_delta
    out_delta = conv_out.copy() * 2
    conv.backward_propagate(out_delta)
    print(conv.filter_grad.shape)
    print(conv.bias_grad.shape)
    print(conv_out.shape)

(2, 32, 32, 12)
(12, 3, 3, 3)
(12,)
(2, 32, 32, 12)
